### Part II. Neural Networks and Deep Learning

# 10. Introduction to Artificial Neural Networks with Keras

Artificial Neural Networks (ANN) is a Machine Learning model inspired by the networks of biological neurons found in our brains.  

Although they draw from our biological brains, they have slightly evolved to be somehow different.

### Why is it different this time?

ANNs have been around for quite some time, with efforts going back to a seminal paper by McCulloch and Pitts (1943). However, after a long winter started in the 1960s they seem to be back in town as the cool kid. 

Why would this time be different? According to the author:

**1. Data quantity**, which allows ANNs to perform traditional ML on large and complex problems

**2. Computing power**, thanks to Moore's Law, the gaming industry (for GPUs) and cloud computing 

**3. Improved algorithms** (not that different from 1990s, but those differences had huge impact)

**4. Theoretical limitations** (e.g. getting stuck in local optima) are **rather rare** in practice, or **not as serious** as previously thought

**5. Virtual cycle** of applications > reaserch + funding > more and better applications

### Logical Computations with Neurons

_**Note**: Skipping the part on biological neurons_

McCulloch and Pitts proposed a simple model of the biological neuron later known as an **artificial neuron** characterized by one or more binary inputs and one binary output. 

Even with this simple neuron, it is possible to build an ANN that computes any logical proposition:

![ANN](images/10.ANN.png)

Assuming an activation threshold of two, we can see from the picture above how this could work. 

### The Perceptron

The next step in complexity is the _Perceptron_. Invented in 1957 by Frank Rosenblatt, it is based on an artificial neuron called a _threshold logic unit_ (TLU) / _linear threshold unit_ (LTU). Inputs and outputs are numbers and each input is connected with a weight.

TLU computes a weighted sum and then applies a step function to the sum:

1. $z = w_1x_1 + w_2n_2 + \cdots + w_nx_n = x^tw$

2. $h_w(x) = step(z) = step(x^tw)$

![Threshold Logic Unit](images/10.TLU.png)

Two common step functions used in Perceptrons:

1. Heaviside (z) $ = \begin{cases}
0 & z < 0\\
1 & z \ge 0
\end{cases}$

2. Sign (z) $ = \begin{cases}
-1 & z < 0 \\
0 & z = 0 \\
+1 & z > 0
\end{cases}$

A single TLU can be used for linear binary classification (using a threshold, similarly to LogReg or SVM).  
A Perceptron is simply a single layer of TLUs, with each TLU connected to all the inputs. Having multiple output TLU makes possible to perform multioutput classification. 

It is then possible to compute the outputs of a layer of artificial neurons for several instances at once:

$h_{W,b} (X) = \phi (XW + b)$

$X$ = matrix of input features  

$W$ = weights of neurons (expept bias). One row per input neuron and one col for artificial neuron. 

$b$ = vector containing all the connection weights between the bias neuron and the artificial neurons. It has one bias term per artificial neuron.

$\phi$ = activation function. In our case (artificial neuron = TLU), the activation function is a step function.

#### Training

The Perceptron is then trained reinforcing connections that help **reduce the error**. 

More specifically, the Perceptron is fed one training instance at a time, and for each instance it makes its predictions. For every output neuron that produced a wrong prediction, it reinforces the connection weights from the inputs that would have contributed to the correct prediction.

More formally:

$w_{i,y}^{(next step)} = w_{i,y} + \eta (y_j - \hat{y}_j)x_i $ 

* $w_{i,y}$ = connection weight between $i^{th}$ input neuron and $j^{th}$ output neuron

* $x_i$ is the $i^{th}$ input value of the current training instance

* $\hat{y}_j$ is the output of the $j^{th}$ output neuron for the current training instance

* $y_j$ is the target output of the $j^{th}$ output neuron for the current training instance

* $\eta$ is the learning rate

**Note**: the Perceptron decision boundaries are linear, but as long as training instances are linearly separable, it will converge to a solution. 

#### Limitations

The Perceptron is a fairly rudimentary ANN architecture, incapable for example to solve the exclusive or (XOR) classification problem. 

It turns out that some of the limitations of Perceptrons can be eliminated by stacking multiple Perceptrons: the resulting ANN is known as **Multilayer Perceptron (MLP)**. 

### Multilayer Perceptron and Backpropagation

An MLP is composed of multiple layers, generally named *input* (lower) layers, *hidden* layers, *output* (upper) layers. 

**Note**: so far we are only dealing with feedforward neural network (FNN). 

Backpropagation comes into play as a training method for MLP. In short, it computes the the gradient of the network’s error with regard to every single model parameter by going forward and then backwards. It does so many times until it converges. 

**Note**: More specifically, automatically computing gradients is called automatic differentiation, or **autodiff**. The autodiff technique used by backpropagation is called **reverse-mode autodiff**. 

Let's run through the algorithm:

1. **Forward pass**: each mini-batch of training instances is passed to the network’s input layer > hidden layers > output layers

2. **Error measurement**: network error computed using a loss function that compares the desired output and the actual output of the network

3. **Error attribution**: it computes how much each output connection contributed to the error

4. **Reverse pass**: tracing back error from output layer to lower layer down to inputs

5. **Gradient Descent**: tweak all the connection weights in the network, using the error gradients computed earlier

**Note**: it is important to randomize the initial hiddent layers weights (_break the simmetry_). 

In order for this to work, David Rumelhart, Geoffrey Hinton, and Ronald Williams replaced the step function with the logistic (sigmoid) function: 

$\displaystyle \sigma(z) = \frac{1}{1+ e^{(-z)}}$

This was particularly helpful for Gradient Descent, since with a stepwise function there is no gradient to work with. Secondly, having a non-linear function (and a chain of non-linear function over many layers) allows us to theoretically approximate any continuous function.

#### Regression MLPs

We can use MLPs for single variable regression tasks (with a single output neuron) or multivariable regression (with multiple output neurons). 

A typical regression MLP architecture consists of the following **hyperparameters**:
* n. Input neurons
* n. Hidden layers
* n. Neurons per hidden layer
* n. Output neurons
* Hidden activation function
* Output activation function
* Loss activation function

#### Classification MLPs

For a binary classification problem, you just need a single output neuron using the logistic activation function: the output will be a number between 0 and 1, which you can interpret as the estimated probability of the positive class. 

MLP classification **hyperparameters**:
* n. Input neurons
* n. Hidden layers
* n. Output neurons
* Output layer activation (log / softmax)
* Loss function

### Implementing MLPs with Keras

[Keras](https://github.com/keras-team/keras) is a high-level Deep Learning API. On the backend, it relies on one of three popular open source Deep Learning libraries: TensorFlow, Microsoft Cognitive Toolkit (CNTK), and Theano.

**Note**: for simplicity we will use the Tensorflow implementation `tf.keras` (without any specific TF features) therefore altough the code in this chapter can be used on any backend implementation (_usually by changing the imports_).

First thing, let's make sure that [Tensorflow](https://www.tensorflow.org/) is up and running (installing it in a virtualenv is recommended):

In [21]:
import tensorflow as tf

In [22]:
from tensorflow import keras

In [24]:
keras.__version__

'2.2.4-tf'

### Building an Image Classifier Using the Sequential API

Let's use the Fashion MNIST dataset:

In [27]:
# loading dataset
fashion_mnist = keras.datasets.fashion_mnist

In [28]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 7s 2us/step


In [29]:
# check shape
X_train_full.shape

(60000, 28, 28)

In [30]:
# check type
X_train_full.dtype

dtype('uint8')

Let's create a validation set. Also, input scales since we will be working with Gradient Descent:

In [31]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0

In [32]:
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [34]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
"Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [35]:
class_names[y_train[0]]

'Coat'

Time to build our Neural Network!

In [36]:
# create Sequential model (simplest NN)
model = keras.models.Sequential()

In [37]:
# first layer - no pars - preprocessing
model.add(keras.layers.Flatten(input_shape=[28, 28]))

In [38]:
# first Dense hidden layer with 300 neurons
model.add(keras.layers.Dense(300, activation="relu"))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [39]:
# second with 100 neurons
model.add(keras.layers.Dense(100, activation="relu"))

In [40]:
# output layer with 10 neurons - softmax act function since we have exclusive classes
model.add(keras.layers.Dense(10, activation="softmax"))

In [41]:
# model summary (default layer names)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In order to _break simmetry_ all the connection weights have been initialized randomly. We can change the initialization method by setting a specific `kernel_initializer`.  

#### Compiling the method

After a model is created, you must call its `compile()` method to specify the loss function and the optimizer to use:

In [42]:
model.compile(loss="sparse_categorical_crossentropy", # because we have sparse labels
              optimizer="sgd", # stochastic gradient descent - generally we would also set a learning rate (def = 0.01)
              metrics=["accuracy"]) # because it's a class problem

#### Training and Evaluation

In [43]:
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/30
55000/55000 [==============================] - 15s 281us/sample - loss: 0.7118 - acc: 0.7629 - val_loss: 0.4968 - val_acc: 0.8374
Epoch 2/30
55000/55000 [==============================] - 12s 213us/sample - loss: 0.4884 - acc: 0.8299 - val_loss: 0.4451 - val_acc: 0.8436
Epoch 3/30
55000/55000 [==============================] - 12s 224us/sample - loss: 0.4434 - acc: 0.8437 - val_loss: 0.4114 - val_acc: 0.8602
Epoch 4/30
55000/55000 [==============================] - 15s 274us/sample - loss: 0.4170 - acc: 0.8538 - val_loss: 0.3982 - val_acc: 0.8650
Epoch 5/30
55000/55000 [==============================] - 12s 218us/sample - loss: 0.3966 - acc: 0.8597 - val_loss: 0.4369 - val_acc: 0.8458
Epoch 6/30
55000/55000 [==============================] - 12s 213us/sample - loss: 0.3800 - acc: 0.8653 - val_loss: 0.3703 - val_acc: 0.8698
Epoch 7/30
55000/55000 [==============================] - 17s 310us/sample - loss: 0.3663 - acc: 0.8693 -

Accuracy **89.30%**, not bad! 

**Note**: if the training set was skewed, it would be useful to set the `class_weight` argument when calling the `fit()` method, which would give a larger weight to underrepresented classes and a lower weight to overrepresented classes.

Now, if we are not satisfied with the performance of our model, there are many things we could do:

* Learning rate
* Change optimizer
* N. of layers
* N. of neurons per layer 
* Activation functions
* Batch size

etc.

Once we are satisfied with **validation** accuracy we can evaluate it on the **test** set. 

In [45]:
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 77us/sample - loss: 77.1612 - acc: 0.8327


[77.16121282806397, 0.8327]

### Make predictions

In [47]:
X_new = X_test[:3]

In [48]:
y_proba = model.predict(X_new)

In [49]:
y_proba.round(2)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

Fairly easy to interpret: e.g. 1 we have a 100% prob of class 10, 0% of any other class. 

### Regression MLP using Sequential API

Let’s switch to the California housing problem and tackle it using a regression neural network. 

In [51]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

INFO:sklearn.datasets.california_housing:Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to C:\Users\giuse\scikit_learn_data


In [52]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

In [53]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [54]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

The main difference here is that we only have one single neuron in the output layer and no activation function.

Since the dataset is quite noisy, we just use a single hidden layer with fewer neurons than before, to avoid overfitting:

In [55]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu",
input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model.predict(X_new)

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 48us/sample - loss: 1.1655 - val_loss: 0.5259
Epoch 2/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.5076 - val_loss: 0.5084
Epoch 3/20
11610/11610 [==============================] - 0s 23us/sample - loss: 0.4671 - val_loss: 0.4615
Epoch 4/20
11610/11610 [==============================] - 0s 24us/sample - loss: 0.4487 - val_loss: 0.4514
Epoch 5/20
11610/11610 [==============================] - 0s 23us/sample - loss: 0.4373 - val_loss: 0.4514
Epoch 6/20
11610/11610 [==============================] - 0s 24us/sample - loss: 0.4417 - val_loss: 0.4289
Epoch 7/20
11610/11610 [==============================] - 0s 23us/sample - loss: 0.4252 - val_loss: 0.4278
Epoch 8/20
11610/11610 [==============================] - 0s 23us/sample - loss: 0.4171 - val_loss: 0.4239
Epoch 9/20
11610/11610 [==============================] - 0s 23us/sample - loss: 0.4111 - val_l

### Building Complex Models Using the Functional API

Although the Sequential API is quite easy to use, we may need to build models with more complex topologies. Here comes the Functional API to the rescue.

One such example is the _wide and deep_ NN, which connects all or part of the inputs directly to the output layer, allowing our network to learn both deep patterns and simple patterns. 

In [56]:
# input object
input_ = keras.layers.Input(shape=X_train.shape[1:]) 
# dense layer with 30 neurons
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
# dense layer with 30 neurons (input is hidden1)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
# concatenate layer (input + hidden2 output)
concat = keras.layers.Concatenate()([input_, hidden2])
# output layer (single layer & no activation function)
output = keras.layers.Dense(1)(concat)
# create the model specifying inputs and outputs
model = keras.Model(inputs=[input_], outputs=[output])

The next step here is to understand how to send a subset of features to the wide net, and a subset to the deep net. To do this, we will use multiple inputs.  

In [57]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

Let's now create our input matrices:

In [58]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]
history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 50us/sample - loss: 1.8344 - val_loss: 0.8213
Epoch 2/20
11610/11610 [==============================] - 0s 28us/sample - loss: 0.7004 - val_loss: 0.6745
Epoch 3/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.6209 - val_loss: 0.6102
Epoch 4/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.5854 - val_loss: 0.5818
Epoch 5/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.5637 - val_loss: 0.5607
Epoch 6/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.5491 - val_loss: 0.5487
Epoch 7/20
11610/11610 [==============================] - 0s 38us/sample - loss: 0.5357 - val_loss: 0.5355
Epoch 8/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.5252 - val_loss: 0.5253
Epoch 9/20
11610/11610 [==============================] - 0s 34us/sample - loss: 0.5168 - val_l

Adding extra outputs to the NN is also quite easy:

In [59]:
# same as above
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])

# connect to appropriate layers
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.Model(inputs=[input_A, input_B], outputs=[output,
aux_output])

Each output will have its own loss function:

In [60]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")

Now when we train the model, we need to provide labels for each output. In this example, the main output and the auxiliary output should try to predict the same thing, so they should use the same labels.

In [61]:
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20,
validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 56us/sample - loss: 0.9901 - main_output_loss: 0.8792 - aux_output_loss: 1.9866 - val_loss: 0.6264 - val_main_output_loss: 0.5634 - val_aux_output_loss: 1.1922
Epoch 2/20
11610/11610 [==============================] - 0s 35us/sample - loss: 0.6089 - main_output_loss: 0.5586 - aux_output_loss: 1.0606 - val_loss: 0.5546 - val_main_output_loss: 0.5068 - val_aux_output_loss: 0.9853
Epoch 3/20
11610/11610 [==============================] - 0s 38us/sample - loss: 0.5288 - main_output_loss: 0.4900 - aux_output_loss: 0.8773 - val_loss: 0.5128 - val_main_output_loss: 0.4751 - val_aux_output_loss: 0.8525
Epoch 4/20
11610/11610 [==============================] - 0s 42us/sample - loss: 0.4969 - main_output_loss: 0.4644 - aux_output_loss: 0.7882 - val_loss: 0.4980 - val_main_output_loss: 0.4665 - val_aux_output_loss: 0.7823
Epoch 5/20
11610/11610 [==============================] - 0s 37us/s

Evaluation using total loss and individual losses:

In [62]:
total_loss, main_loss, aux_loss = model.evaluate(
    [X_test_A, X_test_B], [y_test, y_test])

5160/5160 [==============================] - 0s 23us/sample - loss: 0.3834 - main_output_loss: 0.3689 - aux_output_loss: 0.5004


### Subclassing API for Dynamic Models 

Some models involve loops, varying shapes, conditional branching, and other dynamic behaviors. 

To do that, we can construct a `Model` class, create our layers in the constructor, and use them to perform the computations you want in the `call()` method.

In [66]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # handles standard args (e.g., name)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model = WideAndDeepModel()

This method makes our model much more customizable, but also less easy to inspect. We can also stack up several models together to form more complex models.  

### TensorBoard for Visualization

TensorBoard is an interactive visualization tool useful to: 
* View the learning curves during training
* Compare learning curves between multiple runs
* Visualize the computation graph
* Analyze training statistics
* View images generated by the model
* Visualize complex multidimensional data projected down to 3D and automatically clustered 

.. and more ..

**But**, to use we need to modify our program so that the input is in special binary log files called _event files_. The TensorBoard server will monitor the log
directory, and it will automatically pick up the changes and update the visualizations. 

Ideally, you want the server to point to a root log directory and configure your program so that it writes to a different subdirectory every time it runs.

In [79]:
# defining root log directory
import os
root_logdir = os.path.join(os.curdir, "my_logs")

In [81]:
# create subdir based on time (it may be helpful to add hyperparams and other details)
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [82]:
run_logdir = get_run_logdir() 

Let's start the TensorBoard server. While there is also an option to start it directly from the CLI, I prefer to launch it from Jupyter itself:

In [84]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 18224), started 0:04:00 ago. (Use '!kill 18224' to kill it.)

After this brief introduction on NN, it's time to jump a level deeper!

## Fine-Tuning Neural Network Hyperparameters

One option to find the best hyperparameters is to run random combinations, e.g. using `GridSearchCV`. To do this, we need to wrap our Keras models in objects that mimic regular Scikit-Learn regressors.

In [85]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3,
input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

This function creates a simple Sequential model for univariate regression (only one output neuron), with the given input shape and the given number of hidden layers and neurons, and it compiles it using an SGD optimizer configured with the specified learning rate.

Next, let’s create a `KerasRegressor` based on this `build_model()` function:

In [86]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

Now we can use this object like a regular Scikit-learn regressor: train it with `fit()`, evaluate it with `score()` and use it to make predictions with `predict`:

In [87]:
keras_reg.fit(X_train, y_train, epochs=100,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_new)

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 1s 117us/sample - loss: 1.2916 - val_loss: 0.7770
Epoch 2/100
11610/11610 [==============================] - 1s 53us/sample - loss: 0.6912 - val_loss: 0.6150
Epoch 3/100
11610/11610 [==============================] - 1s 51us/sample - loss: 0.6283 - val_loss: 0.6211
Epoch 4/100
11610/11610 [==============================] - 1s 52us/sample - loss: 0.5408 - val_loss: 0.5206
Epoch 5/100
11610/11610 [==============================] - 1s 49us/sample - loss: 0.5101 - val_loss: 0.5022
Epoch 6/100
11610/11610 [==============================] - 1s 52us/sample - loss: 0.4871 - val_loss: 0.4917
Epoch 7/100
11610/11610 [==============================] - 1s 55us/sample - loss: 0.4747 - val_loss: 0.4806
Epoch 8/100
11610/11610 [==============================] - ETA: 0s - loss: 0.462 - 1s 56us/sample - loss: 0.4646 - val_loss: 0.4865
Epoch 9/100
11610/11610 [==============================] - 1s 

At this point, we can use randomized search to see which variant perform best. Let's focus on # hidden layers, # neurons and learning rate: 

In [89]:
import numpy as np
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10,
cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                validation_data=(X_valid, y_valid),
                callbacks=
[keras.callbacks.EarlyStopping(patience=10)])

Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 1s 121us/sample - loss: 4.2837 - val_loss: 3.0196
Epoch 2/100
7740/7740 [==============================] - 0s 57us/sample - loss: 2.2008 - val_loss: 1.7651
Epoch 3/100
7740/7740 [==============================] - 0s 56us/sample - loss: 1.4705 - val_loss: 1.2920
Epoch 4/100
7740/7740 [==============================] - 1s 68us/sample - loss: 1.1641 - val_loss: 1.0691
Epoch 5/100
7740/7740 [==============================] - 0s 63us/sample - loss: 0.9878 - val_loss: 0.9372
Epoch 6/100
7740/7740 [==============================] - 1s 70us/sample - loss: 0.8804 - val_loss: 0.8587
Epoch 7/100
7740/7740 [==============================] - 1s 86us/sample - loss: 0.8130 - val_loss: 0.8106
Epoch 8/100
7740/7740 [==============================] - 1s 68us/sample - loss: 0.7698 - val_loss: 0.7788
Epoch 9/100
7740/7740 [==============================] - 1s 73us/sample - loss: 0.7402 - val_loss: 0.75

C:\Users\giuse\Anaconda3\lib\site-packages\tensorflow_core\python\keras\callbacks.py:1230: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current - self.min_delta, self.best):


7740/7740 [==============================] - 1s 158us/sample - loss: nan - val_loss: nan
Epoch 4/100
7740/7740 [==============================] - 1s 104us/sample - loss: nan - val_loss: nan
Epoch 5/100
7740/7740 [==============================] - 1s 103us/sample - loss: nan - val_loss: nan
Epoch 6/100
7740/7740 [==============================] - 1s 103us/sample - loss: nan - val_loss: nan
Epoch 7/100
7740/7740 [==============================] - 1s 105us/sample - loss: nan - val_loss: nan
Epoch 8/100
7740/7740 [==============================] - 1s 150us/sample - loss: nan - val_loss: nan
Epoch 9/100
7740/7740 [==============================] - 1s 133us/sample - loss: nan - val_loss: nan
Epoch 10/100
7740/7740 [==============================] - 1s 107us/sample - loss: nan - val_loss: nan
Epoch 11/100
3870/3870 [==============================] - 0s 44us/sample - loss: nan
Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 2s 285us/samp

C:\Users\giuse\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 4s 310us/sample - loss: 1.0443 - val_loss: 0.6549
Epoch 2/100
11610/11610 [==============================] - 3s 301us/sample - loss: 0.5865 - val_loss: 0.5592
Epoch 3/100
11610/11610 [==============================] - 3s 294us/sample - loss: 0.5113 - val_loss: 0.5021
Epoch 4/100
11610/11610 [==============================] - 2s 139us/sample - loss: 0.4637 - val_loss: 0.4609
Epoch 5/100
11610/11610 [==============================] - 2s 145us/sample - loss: 0.4302 - val_loss: 0.4367
Epoch 6/100
11610/11610 [==============================] - 2s 153us/sample - loss: 0.4089 - val_loss: 0.4179
Epoch 7/100
11610/11610 [==============================] - 2s 171us/sample - loss: 0.3952 - val_loss: 0.4082
Epoch 8/100
11610/11610 [==============================] - 3s 241us/sample - loss: 0.3850 - val_loss: 0.3965
Epoch 9/100
11610/11610 [==============================] - 3s 233us/sample - lo

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x0000020A1C4CE708>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020A1A0781C8>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7...
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_sco

It may take a while given the different combinations but once done we will be able to access the best pars, score and trained Keras model:

In [90]:
rnd_search_cv.best_params_

{'learning_rate': 0.003897195998271614, 'n_hidden': 3, 'n_neurons': 76}

In [91]:
rnd_search_cv.best_score_

-0.2999521248973104

In [92]:
model = rnd_search_cv.best_estimator_.model

However, it is worth noting that with large datasets / complex problems this approach does not scale well and allows us to explore only a limited part of the hyperparameters space.  We could manually "zoom in" our search based on the best parameters of the first search but there are also other ways...

Their core idea is simple: when a region of the space turns out to be good, it should be explored more. They do the "zooming in" for us and are more efficient in doing so. 

A few useful libraries:

* Hypeopt
* Hyperas, kopt or Talos
* Keras tuner
* Scikit-Optimize `skopt` (_general purpose_)

and a few more..

The quest for the best hyperparams, best architecture and so on is a very active area of research, contributing among the other things to a more automated approach to Machine Learning (AutoML). 

Despite all this excitement (and hype）it is still useful to know how to do this stuff manually, so let's dig a bit deeper on a few hyperparams tuning.

#### Number of hidden layers

Although in theory 1-layer MLP are already capable of modelling increadibly complex functions, they are not very **parameter efficient** (capacity to model omplex functions using
exponentially fewer neurons than shallow nets > better performance with less training data).

In addition to the point mentioned above, it's also worth noting that most of the time we deal with complex tasks we will actually reuse existing state of the art network rather than train one from scratch. 

#### Number of neurons per hidden layer

**I/O** layers are usually determined by the task at hand. For MNIST, we need to input 28 x 28 = 784 pixels and output 10 values (likelihood of being a number from 0 to 9).

**Hidden**: it used to be common practice to decrease the number of neuron per hidden layer progressively, but it has been found that an equal number of neuron in each layer doesn't impact performance significantly.

